In [1]:
from azurewrapper.workspace import get_workspace

# subscription_id = "29d66431-a7ce-4709-93f7-3bdb01a243b3"
# resource_group = "ExperimentationJayke"
# workspace_name = "ExperimentationJayke"
subscription_id = "a00eaec6-b320-4e7c-ae61-60a30aec1cfc"
resource_group = "MachineLearning"
workspace_name = "RiverImageAnalysis"
tenant_id = "86f9fea7-9eb0-4325-8b58-7ed0db623956"

workspace = get_workspace(subscription_id, resource_group, workspace_name, tenant_id=tenant_id)

In [2]:
from azureml.core import Model

model = Model(workspace, "yolo_v4")

In [3]:
from azurewrapper.environment import get_environment

environment = get_environment(
    workspace,
    "tensorflow-yolo",
    pip_requirements="./examples/yolo/Tensorflow_YOLO/requirements.txt",
    docker_image="mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04"
)

No environment with that name found, creating new one


In [11]:
from azurewrapper.deploy import deploy
service = deploy(
    workspace,
    "testdeploy1",
    model,
    "score.py",
    "examples/yolo",
    environment=environment,
)
service.wait_for_deployment(show_output = True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0ea7be47d3e548c3bd0de28701c8bddf.azurecr.io
Logging into Docker registry 0ea7be47d3e548c3bd0de28701c8bddf.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0ea7be47d3e548c3bd0de28701c8bddf.azurecr.io/azureml/azureml_ea71d6d6fbafdaeb54907902a21c77dc
 ---> 858c385d508a
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> e838e06b60fd
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImEwMGVhZWM2LWIzMjAtNGU3Yy1hZTYxLTYwYTMwYWVjMWNmYyIsInJlc291cmNlR3JvdXBOYW1lIjoibWFjaGluZWxlYXJuaW5nIiwiYWNjb3VudE5hbWUiOiJyaXZlcmltYWdlYW5hbHlzaXMiLCJ3b3Jrc3BhY2VJZCI6IjBlYTdiZTQ3LWQzZTUtNDhjMy1iZDBkLWUyODcwMWM4YmRkZiJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 21788c2e63c6
 ---> 4e8c6a820abf
Step 4/5 : RUN mv '/var/azureml-app/tmphyluykb3.py' /var/azureml-app/main.py
 ---> Running in

ERROR - Error: Container has crashed. Did your init method fail?




Container Logs:
2020-11-12T16:38:26,530762818+00:00 - gunicorn/run 
2020-11-12T16:38:26,530462418+00:00 - iot-server/run 
2020-11-12T16:38:26,530775186+00:00 - rsyslog/run 
2020-11-12T16:38:26,804977295+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2020-11-12T16:38:31,177043078+00:00 - iot-server/finish 1 0
2020-11-12T16:38:31,179233721+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (7)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 45
2020-11-12 16:38:38.591506: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2020-11-12 16:39:01,696 | root | INFO | Starting up app insights client
Starting up app insights client
2020-11-12 16:39:01,696 | root | INFO | Starting up request id generator
Starting up requ

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [41]:
service.reload()

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.


In [39]:
service.get_logs(num_lines=10000)

'2020-11-12T17:15:27,624050294+00:00 - iot-server/run \n2020-11-12T17:15:27,625413175+00:00 - rsyslog/run \n2020-11-12T17:15:27,627167282+00:00 - gunicorn/run \n2020-11-12T17:15:27,629166198+00:00 - nginx/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\nStarting gunicorn 19.9.0\nListening at: http://127.0.0.1:31311 (12)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 41\n2020-11-12T17:15:28,758996899+00:00 - iot-server/finish 1 0\n2020-11-12T17:15:28,760590561+00:00 - Exit code 1 is normal. Not restarting iot-server.\n2020-11-12 17:15:29.076269: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1\nSPARK_HOME not set. Skipping PySpark Initialization.\nInitializing logger\n2020-11-12 17:15:30,493 | root | INFO | Starting up app insights client\nStarting up app insights client\n2020-11-12 17:15:30,494 | root | INFO | Starting up request id generator\nStarting up r

In [42]:
service.delete()

Container has been successfully cleaned up.
